# Lattice Fermions, from Slater Determinants to Neural Backflow Transformations

Author: Giuseppe Carleo (Computational Quantum Science Lab - EPFL) 

In this tutorial, we will introduce you to studying fermionic quantum many-body systems using NetKet. We will start by introducing fermionic operators and how to work with them in NetKet. We will then proceed to implement and optimize two different wave functions: a Slater determinant wave function and a Slater Backflow neural wave function, for a model of spinless fermions in two dimensions. In the following we will assume the reader is familiar with the main concepts in second quantization, including creation and destruction operators, as well as the role of anticommutation relations. 

## Fermionic Operators in NetKet`

Fermionic operators are fundamental to describing quantum systems with fermions (e.g., electrons). NetKet provides tools to work with these operators efficiently. Let's start by setting up the necessary environment and defining our quantum system. 
We start by importing the main netket library, as well as the experimental part of it. The latter is where fermionic operators are currently contained. In the next major release of netket this will change. 

In [1]:
import netket as nk
import netket.experimental as nkx

We will work with a Hamiltonian of spinless fermions on a two-dimensional lattice: 

$$
\mathcal{H} = -t \sum_{\langle i,j \rangle } \left( c^{\dagger}_i c_j + c^{\dagger}_j c_i \right ) + V \sum_{\langle i,j \rangle } n_i n_j 
$$

Here, $\langle i,j \rangle $ denotes nearest-neighbors on a square lattice of $L\times L$ sites, $c_i (c^{\dagger}_i)$ are destruction (creation) fermionic operators on site $i$, whereas $n_i=c^{\dagger}_i c_i$ are density operators. 

## Defining the lattice and the Hilbert space: 

In [2]:
L = 4  # Side of the square
graph = nk.graph.Square(L)
N = graph.n_nodes

In [3]:
N

16

The last variable contains the total number of sites on the lattice. 

We now define also the Hilbert space associated with a system of $N_{\mathrm{f}}$ spinless fermions: 

In [4]:
N_f = 5

hi = nkx.hilbert.SpinOrbitalFermions(N, s=None, n_fermions=N_f)

## Fermionic Operators and Hamiltonian

To describe the Hamiltonian of our quantum system, we need to create fermionic operators. These operators include creation (cdag), annihilation (c), and number (nc) operators. We will use these operators to build our Hamiltonian.

In [5]:
def c(site):
    return nkx.operator.fermion.destroy(hi, site)

def cdag(site):
    return nkx.operator.fermion.create(hi, site)

def nc(site):
    return nkx.operator.fermion.number(hi, site)

With these operators, we can now construct the Hamiltonian for our system. In this example, we have a tight-binding hopping term proportional to $t$ and a density-density interaction term proportional to $V$. We can easily define the Hamiltonian by adding terms one by one looping over the edges of the lattice: 


In [6]:
t = 1.0
V = 4.0

H = 0.0
for (i, j) in graph.edges():
    H -= t * (cdag(i) * c(j) + cdag(j) * c(i))
    H += V * nc(i) * nc(j)

## Exact Diagonalization 

Since the system is relatively small, the Hilbert space is also not too big, and we can still use exact diagonalization to compute the ground state energy. This is achieved by first converting the Hamiltonian to a sparse matrix, and then diagonalizing it with scipy: 

In [7]:
# Convert the Hamiltonian to a sparse matrix
sp_h = H.to_sparse()

In [8]:
from scipy.sparse.linalg import eigsh

eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")

print("Exact ground state energy:", eig_vals[0])

Exact ground state energy: -6.859013554319571


## Slater Determinant

Now, let's move on to defining and optimizing a simple variational wave function based on a mean-field state: the Slater determinant. 

Formally, we can write the state as filling up $N_{\mathrm{f}}$ orbitals: 

$$
|\Phi_s\rangle = \Pi_{\alpha=1}^{N_{\mathrm{f}}} \phi^{\dagger}_{\alpha} |0\rangle,
$$

where $0\rangle$ is the vacuum state and the single-particle orbitals are created by the operators $\phi^{\dagger}_{\alpha}$. In turn, these creation operators are, in general, a linear combination of the original creation operators:

$$
\phi^{\dagger}_{\alpha} = \sum_i M_{i,\alpha} c^{\dagger}_i.
$$

The rectangular ($N\times N_{\mathrm{f}}$) matrix $M$ constitutes a set of free variational parameters. 

It can be shown that the amplitudes of the wave function in the computational basis $|n_1,\dots,n_N\rangle$ are determinants:

$$
\langle n_1,\dots,n_N |\Phi_s\rangle = \mathrm{det}\left\{A(n)\right\},
$$

where the $N_{\mathrm{e}}\times N_{\mathrm{e}}$ matrix is

$$
A(n)_{\alpha,\beta} = M_{R_i,\beta},
$$

where $R_i(n)$ denotes the sites that are occupied, compatible with the occupation numbers $n=(n_1,\dots,n_L)$. For more details see Chapter 5 of Reference [1]. 

To write down this variational amplitudes, we start by defining a convenience function to compute the logarithm of the determinant of a matrix, in the complex domain, and using jax:

In [9]:
import jax
import jax.numpy as jnp

def _log_det(phi):
    log_det = jnp.linalg.slogdet(phi)
    return jnp.asarray(jnp.log(log_det[0].astype(complex)) + log_det[1].astype(complex)).astype(complex)

Next, we define a wave function using Flax. As you might have seen also in other Tutorials, NetKet defines the logarithm of the wave function amplitudes, mostly to avoid overflow/underflow when computing relevant quantities. The model wave function is then: 

In [10]:
import flax.linen as nn
from netket.utils.types import NNInitFunc
from netket.nn.masked_linear import default_kernel_init
from typing import Any, Callable, Sequence
from functools import partial
DType = Any


class LogSlaterDeterminant(nn.Module):
    hilbert: nkx.hilbert.SpinOrbitalFermions
    kernel_init: NNInitFunc = default_kernel_init
    param_dtype: DType = float

    @nn.compact
    def __call__(self, n):
        kernel = self.param('M', self.kernel_init, (self.hilbert.n_fermions, self.hilbert.n_orbitals,), self.param_dtype)

        @partial(jnp.vectorize, signature='(n)->()')
        def log_sd(n):
            phi = kernel[:, jnp.where(n, size=self.hilbert.n_fermions)[0]]
            return _log_det(phi)

        return log_sd(n)

This Flax module defines the variational parameters to be the rectangular matrix $M$. In gneral, these parameters can be real or complex valued. In the following we will work with real parameters, for simplicity. 

## Creating and Optimizing the Slater Determinant Wave Function

We now create an instance of the `LogSlaterDeterminant` class and of a suitable Monte Carlo Sampler to obtain expected values using Variational Monte Carlo:

In [11]:
# Create the Slater determinant model
model = LogSlaterDeterminant(hi)

# Define the Metropolis-Hastings sampler
sa = nk.sampler.MetropolisExchange(hi, graph=graph)


Here we use a sampler that exchanges the occupation numbers of two sites. This allows to keep the total number of fermions constant. 

We also define the `VariationalState` necessary to compute expected values over the variational state using Monte Carlo sampling. We will use a total of 16 independent Markov Chains and $2^{12}$ samples per chain. We will also discard the first 16 samples of each chain, to allow thermalization: 

In [12]:
vstate = nk.vqs.MCState(sa, model, n_samples=2**12, n_discard_per_chain=16)

For example, we can generate samples distributed according to the square modulus of our variational state, and check its shape:

In [13]:
vstate.samples.shape

(16, 256, 16)

You see here that the first index corresponds to the number of chain, the second to the samples on each chain, and the last one is the index of the occupation number, for example one configuration sampled looks like:

In [14]:
vstate.samples[0,0]

Array([0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1.],      dtype=float64)

As you can see, everything is compatible with what you specified in the Hilbert space, namely there are exactly $N_{\mathrm{f}}=5$ non-zero occupation numbers.

Then, we can proceed and optimize for the ground state wave function, defining a suitable optimizer and, in this case, also a preconditioner based on the Quantum Natural Gradient (or Stochastic Reconfiguration):

In [15]:
# Define the optimizer
op = nk.optimizer.Sgd(learning_rate=0.05)

# Define a preconditioner
preconditioner = nk.optimizer.SR(diag_shift=0.05, holomorphic=False)

# Create the VMC (Variational Monte Carlo) driver
gs = nk.VMC(H, op, variational_state=vstate, preconditioner=preconditioner)


A more detailed explanation of `SR` can be found in the Documentation, here we just comment on the fact that we have specified the keyword `holomorphic=False`, since our wave function has complex-valued output but real paramaters, thus it is non holomorphic strictu sensu.

We can now finally optimize the wave function for 300 steps of VMC: 

In [16]:
# Run the optimization for 300 iterations
gs.run(n_iter=300, out="test")

100%|███████████████████████████████████████████████████████████| 300/300 [02:32<00:00,  1.97it/s, Energy=-5.060-0.000j ± 0.041 [σ²=4.173, R̂=1.0056]]


(JsonLog('test', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.2461397647857666
   	Params: 0.003977537155151367,)

After optimizating the wave function, we can evaluate the energy on the final set of paramaters and compare to the exact value:

In [17]:
sd_energy = vstate.expect(H)
error = abs((sd_energy.mean - eig_vals[0]) / eig_vals[0])

print(f"Optimized energy : {sd_energy}")
print(f"Relative error   : {error}")

Optimized energy : -5.017+0.000j ± 0.041 [σ²=4.146, R̂=1.0027]
Relative error   : 0.26860801399545686


As you can see, the mean field energy of the Slater Determinant is about 25% off in this case where interactions are strong, thus far from the single-particle regime in which the Slater Determinant is accurate.

## Creating and Optimizing the Neural-Backflow Wave Function

In order to go beyond the simple mean field approximation, we make use of the Neural Backflow transformation of Reference [4].  
The idea is to promote the matrix $M$ to be a function of all the occupation numbers, through a neural network. Specifically, we take an additive form of the backflow transformation:

$$
M^{\mathrm{bf}}_{i,\alpha}(n) = M_{i,\alpha} + F_{i,\alpha}(n)
$$

and parameterize $F$ with a multilayer perceptron taking $N_{\mathrm{f}}$ inputs and oupting $N\times N_{\mathrm{f}}$ numbers. 

In [18]:
class LogSlaterBfDeterminant(nn.Module):
    hilbert: nkx.hilbert.SpinOrbitalFermions
    hidden_units: int
    kernel_init: NNInitFunc = default_kernel_init
    param_dtype: DType = float

    @nn.compact
    def __call__(self, n):
        assert self.hilbert.spin is None

        @partial(jnp.vectorize, signature='(n)->()')
        def log_sd(n):
            kernel = self.param('Phi', self.kernel_init, (self.hilbert.n_fermions, self.hilbert.n_orbitals,), self.param_dtype)
            bf = nn.Dense(self.hidden_units, param_dtype=self.param_dtype)(n)
            bf = jax.nn.tanh(bf)
            bf = nn.Dense(self.hilbert.n_orbitals * self.hilbert.n_fermions, param_dtype=self.param_dtype)(bf).reshape(kernel.shape)
            kernel += bf
            phi = kernel[:, jnp.where(n, size=self.hilbert.n_fermions)[0]]
            return _log_det(phi)

        return log_sd(n)

We can then proceed as above to optimize this variational state using VMC.

In [19]:
# Create a model using the chosen module (e.g., LogSlaterDeterminant or LogSlaterBfDeterminant)
model = LogSlaterBfDeterminant(hi, hidden_units=N, param_dtype=float)

# Define a Metropolis exchange sampler
sa = nk.sampler.MetropolisExchange(hi, graph=graph)

# Define an optimizer
op = nk.optimizer.Sgd(learning_rate=0.05)

# Create a variational state
vstate = nk.vqs.MCState(sa, model, n_samples=2**12, n_discard_per_chain=16)

# Create a Variational Monte Carlo driver
preconditioner = nk.optimizer.SR(diag_shift=0.05, holomorphic=False)
gs = nk.VMC(H, op, variational_state=vstate, preconditioner=preconditioner)

# Run the optimization for 300 iterations
gs.run(n_iter=300, out="test")

100%|███████████████████████████████████████████████████████████| 300/300 [04:11<00:00,  1.19it/s, Energy=-6.832-0.000j ± 0.012 [σ²=0.499, R̂=1.0016]]


(JsonLog('test', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.21848750114440918
   	Params: 0.007473945617675781,)

We can further check how good the optimized energy is:

In [20]:
sd_energy = vstate.expect(H)
error = abs((sd_energy.mean - eig_vals[0]) / eig_vals[0])

print(f"Optimized energy : {sd_energy}")
print(f"Relative error   : {error}")

Optimized energy : -6.818+0.000j ± 0.010 [σ²=0.450, R̂=1.0014]
Relative error   : 0.005950436716107762


Thus, as expected, the Neural Backflow achieves a significantly higher level of precision (0.5%) versus the 25% error of the mean field state. The result can be further improved by playing with the feedforward architecture defining the backflow, for example by increasing 'hidden_units' or by improving the optimization increasing the number of samples and/or the number of steps. 

# References

[1] Becca, F. & Sorella, S. Quantum Monte Carlo Approaches for Correlated Systems. (Cambridge University Press, 2017).

[2] Nomura, Y., Darmawan, A. S., Yamaji, Y. & Imada, M. Restricted Boltzmann machine learning for solving strongly correlated quantum systems. Phys. Rev. B 96, 205152 (2017).

[3] Stokes, J., Moreno, J. R., Pnevmatikakis, E. A. & Carleo, G. Phases of two-dimensional spinless lattice fermions with first-quantized deep neural-network quantum states. Phys. Rev. B 102, 205122 (2020).

[4] Luo, D. & Clark, B. K. Backflow Transformations via Neural Networks for Quantum Many-Body Wave Functions. Phys. Rev. Lett. 122, 226401 (2019).

